In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from IPython.display import display, HTML

# Test display
display(HTML("<b>IPython display is working!</b>"))


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\User\Desktop\AI_Lab\WK4-Labs\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [5]:
# Your code

import pandas as pd
from sklearn.model_selection import train_test_split

# Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(
    r"C:\Users\User\Desktop\AI_Lab\WK4-Labs\lab-natural-language-processing\data\kg_train.csv",
    encoding='latin-1'
)

# Reduce the training set to speed up development
# Modify for final system
data = data.head(1000)
print("Dataset shape:", data.shape)

# Fill missing values with empty strings
data.fillna("", inplace=True)

# Assuming 'EmailText' is the column containing the message and 'Label' the target
X = data['text']
y = data['label']

# Divide into training and test partitions (e.g., 80% train / 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Quick check
print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])


Dataset shape: (1000, 2)
Training set size: 800
Test set size: 200


In [24]:
print(data.columns)



Index(['text', 'label'], dtype='object')


In [ ]:
# Example: Suppose the columns are actually 'text' and 'label'
X = data['text']     
y = data['label']       

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])


Training set size: 800
Test set size: 200


## Data Preprocessing

In [7]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
# Your code

import re
from html import unescape

# Patterns
SCRIPT_STYLE_RE = re.compile(r"(?is)<(script|style)[^>]*>.*?</\1>", re.MULTILINE)
COMMENTS_RE     = re.compile(r"(?is)<!--.*?-->", re.MULTILINE)
TAGS_RE         = re.compile(r"(?is)<[^>]+>")

def clean_html_regex(html: str) -> str:
    # 1) Remove inline JS/CSS
    s = SCRIPT_STYLE_RE.sub(" ", html)
    # 2) Remove comments
    s = COMMENTS_RE.sub(" ", s)
    # 3) Remove remaining tags
    s = TAGS_RE.sub(" ", s)
    # 4) Unescape & normalize
    s = unescape(s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- Define demo HTML ---
html_doc = """
<html>
<head>
<style>.red {color: red;}</style>
<script>alert('hi');</script>
</head>
<body>
<!-- comment with > tricky char -->
<p>Hello <b>World</b> &amp; welcome!</p>
</body>
</html>
"""

# Test
print(clean_html_regex(html_doc))
# Output: "Hello World & welcome!"



Hello World & welcome!


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
# Your code

import re

def clean_text(s):
    """
    Clean text by applying (in order):
      1) Remove special characters
      2) Remove numbers
      3) Remove all single characters
      4) Remove single characters from the start
      5) Substitute multiple spaces with a single space
      6) Remove prefixed 'b' (from byte-string representations)
      7) Convert to lowercase
    """
    # Ensure string (handle bytes safely)
    if isinstance(s, bytes):
        s = s.decode('utf-8', errors='ignore')
    else:
        s = str(s)

    # 6) Remove prefixed 'b' if present like b'...'
    #    (Do this early so quotes don't interfere with later steps)
    #    Matches:  b'...content...'  or  b"...content..."
    m = re.match(r"^\s*b([\"'])(.*)\1\s*$", s)
    if m:
        s = m.group(2)

    # 7) Convert to lowercase
    s = s.lower()

    # 1) Remove special characters (keep letters and spaces only)
    #    Note: If you want to keep punctuation, adjust this.
    s = re.sub(r"[^a-z\s]", " ", s)

    # 2) Remove numbers (already excluded above, but keep explicitly)
    s = re.sub(r"\d+", " ", s)

    # 3) Remove all single characters (isolated letters like 'a', 'I')
    s = re.sub(r"\b[a-z]\b", " ", s)

    # 4) Remove single characters from the start of the string
    s = re.sub(r"^\s*\b[a-z]\b\s*", " ", s)

    # 5) Substitute multiple spaces with single space
    s = re.sub(r"\s+", " ", s).strip()

    return s

# ---- Demo ----
examples = [
    "b'This is a TEST! With #hashtags, numbers 123 and single letters a I t.'",
    b"Bytes input: Hello!! 42 times... a b c?",
    "  x  Leading single letters should go.  ",
]
for e in examples:
    print("IN :", repr(e))
    print("OUT:", clean_text(e), "\n")


IN : "b'This is a TEST! With #hashtags, numbers 123 and single letters a I t.'"
OUT: this is test with hashtags numbers and single letters 

IN : b'Bytes input: Hello!! 42 times... a b c?'
OUT: bytes input hello times 

IN : '  x  Leading single letters should go.  '
OUT: leading single letters should go 



## Now let's work on removing stopwords
Remove the stopwords.

In [10]:
# Your code

import re
import nltk
from nltk.corpus import stopwords

# Download stopwords once
nltk.download('stopwords')

# Load English stopwords into a set for fast lookup
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    """
    Remove English stopwords from a given text string.
    Assumes the text is already lowercased and cleaned.
    """
    # Tokenize into words (split on whitespace)
    words = text.split()
    
    # Keep words not in stopwords
    filtered_words = [w for w in words if w not in stop_words]
    
    # Join back into a string
    return " ".join(filtered_words)

# Clean the text first
data["clean_text"] = data["text"].apply(clean_text)  # Or clean_html_regex if HTML cleaning


# --- Demo ---
example = "this is an example sentence showing off the stop words filtration"
print("Original:", example)
print("No stopwords:", remove_stopwords(example))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Original: this is an example sentence showing off the stop words filtration
No stopwords: example sentence showing stop words filtration


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [15]:
# Your code

import nltk

# Tokenizer
nltk.download('punkt')

# POS tagger (name changed in newer NLTK)
# Try both so it works across versions:
for pkg in ['averaged_perceptron_tagger', 'averaged_perceptron_tagger_eng']:
    try:
        nltk.download(pkg)
    except Exception as e:
        print(f"Could not download {pkg}: {e}")

# Lemmatizer data
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize

def _ensure_nltk_data():
    needed = [
        ('tokenizers/punkt', 'punkt'),
        ('corpora/wordnet', 'wordnet'),
        ('corpora/omw-1.4', 'omw-1.4'),
    ]
    # POS tagger: handle both names across NLTK versions
    pos_candidates = [
        ('taggers/averaged_perceptron_tagger', 'averaged_perceptron_tagger'),
        ('taggers/averaged_perceptron_tagger_eng', 'averaged_perceptron_tagger_eng'),
    ]
    for path, pkg in needed + pos_candidates:
        try:
            nltk.data.find(path)
        except LookupError:
            nltk.download(pkg, quiet=True)

_ensure_nltk_data()

lemmatizer = WordNetLemmatizer()

def _wn_pos(tag):
    if tag.startswith('J'): return wordnet.ADJ
    if tag.startswith('V'): return wordnet.VERB
    if tag.startswith('N'): return wordnet.NOUN
    if tag.startswith('R'): return wordnet.ADV
    return wordnet.NOUN

def lemmatize_text(text: str) -> str:
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    lemmas = [lemmatizer.lemmatize(w, _wn_pos(t)) for w, t in tagged]
    return " ".join(lemmas)

# Test
print(lemmatize_text("The cats are running faster than the dogs."))


The cat be run faster than the dog .


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [17]:
# Your code

import pandas as pd
from collections import Counter
import re

# --- Assumes you have a DataFrame `data`
# Replace these with your actual column names:
text_col = "text"   # column with the message text
label_col = "label"      # column with 'ham'/'spam' or 0/1

# Simple tokenizer (lowercase, keep only letters)
def tokenize(text):
    text = text.lower()
    words = re.findall(r'\b[a-z]{2,}\b', text)  # only words with >=2 letters
    return words

# Separate ham and spam messages
ham_texts = data[data[label_col].isin(["ham", 0])][text_col].apply(tokenize)
spam_texts = data[data[label_col].isin(["spam", 1])][text_col].apply(tokenize)

# Flatten token lists
ham_words = [word for tokens in ham_texts for word in tokens]
spam_words = [word for tokens in spam_texts for word in tokens]

# Count and get top 10
top_ham = Counter(ham_words).most_common(10)
top_spam = Counter(spam_words).most_common(10)

# Display
print("Top 10 words in HAM messages:")
for word, freq in top_ham:
    print(f"{word:15s} {freq}")

print("\nTop 10 words in SPAM messages:")
for word, freq in top_spam:
    print(f"{word:15s} {freq}")


Top 10 words in HAM messages:
the             1773
to              1065
and             833
of              791
in              616
that            414
is              385
for             369
on              329
you             311

Top 10 words in SPAM messages:
the             7046
to              5593
of              4984
and             3985
in              3289
you             3229
this            2675
my              2143
your            2078
for             2030


In [21]:
# Example cleaning function
def clean_text_basic(text):
    # Lowercase
    text = text.lower()
    # Remove special characters/numbers
    text = re.sub(r"[^a-z\s]", "", text)
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 1. Create 'preprocessed_text' before splitting
data["preprocessed_text"] = data["text"].apply(clean_text_basic)

# 2. Split into train/val
from sklearn.model_selection import train_test_split
data_train, data_val = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
    stratify=data['label']  # replace with your actual target column
)

data_train = data_train.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)

# 3. Add extra features
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", r"\$"])
suspicious_words = "|".join([
    "free", "cheap", "sex", "money", "account", "bank", "fund",
    "transfer", "transaction", "win", "deposit", "password"
])

for df in [data_train, data_val]:
    df['money_mark'] = df['preprocessed_text'].str.contains(money_simbol_list, case=False, regex=True) * 1
    df['suspicious_words'] = df['preprocessed_text'].str.contains(suspicious_words, case=False, regex=True) * 1
    df['text_len'] = df['preprocessed_text'].apply(len)

print(data_train.head())


                                                text  label  \
0  Dear=2C Good day hope fine=2Cdear am writting ...      1   
1  FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...      1   
2                                           Will do.      0   
3  FROM THE DESK OF DR.ADAMU  ISMALERAUDITING AND...      1   
4  Dear Friend, My name is LOI C.ESTRADA,The wife...      1   

                                          clean_text  \
0  dear good day hope fine cdear am writting this...   
1  from mr henry kaborethe chief auditor incharge...   
2                                            will do   
3  from the desk of dr adamu ismalerauditing and ...   
4  dear friend my name is loi estrada the wife of...   

                                   preprocessed_text  money_mark  \
0  dearc good day hope finecdear am writting this...           1   
1  from mr henry kaborethe chief auditor incharge...           0   
2                                            will do           0   
3  from the 

## Extra features

In [22]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).

money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,clean_text,preprocessed_text,money_mark,suspicious_words,text_len
0,Dear=2C Good day hope fine=2Cdear am writting ...,1,dear good day hope fine cdear am writting this...,dearc good day hope finecdear am writting this...,1,1,1558
1,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,from mr henry kaborethe chief auditor incharge...,from mr henry kaborethe chief auditor incharge...,0,1,2936
2,Will do.,0,will do,will do,0,0,7
3,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,from the desk of dr adamu ismalerauditing and ...,from the desk of dradamu ismalerauditing and a...,1,1,497
4,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,dear friend my name is loi estrada the wife of...,dear friend my name is loi cestradathe wife of...,1,1,2050


In [23]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# --- 0) Locate columns ---
def detect_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    raise KeyError(f"None of these columns found: {candidates}")

# Adjust candidate lists if needed
text_col  = detect_col(
    set(data.columns),
    ["preprocessed_text","EmailText","text","message","content","sms","Email","body"]
)
label_col = detect_col(
    set(data.columns),
    ["label","Label","target","y","spam","is_spam"]
)

# --- 1) Minimal text cleaner to build `preprocessed_text` if missing ---
def basic_clean(s: str) -> str:
    s = str(s)
    s = s.lower()
    # remove urls, mentions, digits
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"@\w+|#\w+", " ", s)
    s = re.sub(r"\d+", " ", s)
    # strip html tags
    s = re.sub(r"(?is)<(script|style)[^>]*>.*?</\1>", " ", s)
    s = re.sub(r"(?is)<!--.*?-->", " ", s)
    s = re.sub(r"(?is)<[^>]+>", " ", s)
    # keep letters and spaces only
    s = re.sub(r"[^a-z\s]", " ", s)
    # collapse spaces
    s = re.sub(r"\s+", " ", s).strip()
    return s

if "preprocessed_text" not in data.columns:
    data["preprocessed_text"] = (
        data[text_col].fillna("").astype(str).apply(basic_clean)
    )

# --- 2) Split into train/val ---
data_train, data_val = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
    stratify=data[label_col] if data[label_col].nunique() > 1 else None
)
data_train = data_train.reset_index(drop=True)
data_val   = data_val.reset_index(drop=True)

# --- 3) Extra features (your indicators + a couple helpful ones) ---
def add_extra_features(df: pd.DataFrame, text_col: str = "preprocessed_text") -> pd.DataFrame:
    df[text_col] = df[text_col].fillna("").astype(str)

    # your original patterns (case-insensitive)
    money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
    suspicious_words  = "|".join([
        "free","cheap","sex","money","account","bank","fund","transfer",
        "transaction","win","deposit","password"
    ])

    df['money_mark'] = df[text_col].str.contains(money_simbol_list, case=False, regex=True, na=False).astype('int8')
    df['suspicious_words'] = df[text_col].str.contains(suspicious_words, case=False, regex=True, na=False).astype('int8')
    df['text_len'] = df[text_col].str.len().astype('int32')

    # extras
    df['token_count']  = df[text_col].str.split().apply(len).astype('int32')
    df['exclam_count'] = df[text_col].str.count(r"!").astype('int16')
    return df

data_train = add_extra_features(data_train, text_col="preprocessed_text")
data_val   = add_extra_features(data_val,   text_col="preprocessed_text")

print(data_train[[text_col if text_col!='preprocessed_text' else 'preprocessed_text',
                  label_col,'money_mark','suspicious_words','text_len','token_count','exclam_count']].head())

# (Optional) sanity check:
# print("Columns:", list(data.columns))


                                   preprocessed_text  label  money_mark  \
0  dearc good day hope finecdear am writting this...      1           1   
1  from mr henry kaborethe chief auditor incharge...      1           0   
2                                            will do      0           0   
3  from the desk of dradamu ismalerauditing and a...      1           1   
4  dear friend my name is loi cestradathe wife of...      1           1   

   suspicious_words  text_len  token_count  exclam_count  
0                 1      1558          286             0  
1                 1      2936          492             0  
2                 0         7            2             0  
3                 1       497           76             0  
4                 1      2050          320             0  


## How would work the Bag of Words with Count Vectorizer concept?

In [25]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Example small corpus (list of text documents)
corpus = [
    "I love natural language processing",
    "Language processing is fun",
    "I love coding in Python"
]

# Step 1: Initialize CountVectorizer
vectorizer = CountVectorizer()

# Step 2: Fit the vectorizer and transform the corpus into BoW matrix
X_bow = vectorizer.fit_transform(corpus)

# Step 3: Inspect vocabulary
print("Vocabulary:", vectorizer.get_feature_names_out())

# Step 4: Convert to DataFrame for better readability
bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())
print("\nBag of Words Representation:\n", bow_df)


Vocabulary: ['coding' 'fun' 'in' 'is' 'language' 'love' 'natural' 'processing'
 'python']

Bag of Words Representation:
    coding  fun  in  is  language  love  natural  processing  python
0       0    0   0   0         1     1        1           1       0
1       0    1   0   1         1     0        0           1       0
2       1    0   1   0         0     1        0           0       1


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [30]:
# Your code

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Download stopwords once
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Example dataset (replace with your actual text column)
data = pd.DataFrame({
    "text": [
        "I love natural language processing",
        "Language processing is fun and interesting",
        "I love coding in Python for data science",
        "Python and data processing are amazing"
    ]
})

# Step 1: Load the vectorizer
# Includes stopword removal, unigrams + bigrams, and a vocabulary size limit
vectorizer = TfidfVectorizer(
    stop_words=stop_words,
    ngram_range=(1, 2),
    max_features=20
)

# Step 2: Vectorize the entire dataset
X_tfidf = vectorizer.fit_transform(data["text"])

# Step 3: Print shape
print("Shape of TF-IDF matrix:", X_tfidf.shape)

# (Optional) Convert to DataFrame to inspect
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
print("\nTF-IDF DataFrame:\n", tfidf_df.head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Shape of TF-IDF matrix: (4, 20)

TF-IDF DataFrame:
     amazing    coding  coding python      data  data processing  data science  \
0  0.000000  0.000000       0.000000  0.000000         0.000000      0.000000   
1  0.000000  0.000000       0.000000  0.000000         0.000000      0.000000   
2  0.000000  0.426931       0.426931  0.336597         0.000000      0.426931   
3  0.435516  0.000000       0.000000  0.343366         0.435516      0.000000   

        fun  fun interesting  interesting  language  language processing  \
0  0.000000         0.000000     0.000000  0.343366             0.343366   
1  0.420681         0.420681     0.420681  0.331670             0.331670   
2  0.000000         0.000000     0.000000  0.000000             0.000000   
3  0.000000         0.000000     0.000000  0.000000             0.000000   

       love  love natural   natural  natural language  processing  \
0  0.343366      0.435516  0.435516          0.435516    0.277984   
1  0.000000      0.0000

## And the Train a Classifier?

In [54]:
# Your code

import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# ---------- helpers ----------
def detect_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

def guess_label_col(df: pd.DataFrame, text_like=("text","message","content","email","body","sms")):
    """
    Try to guess a binary label column:
      - exclude text-like columns
      - pick a column with exactly 2 unique values (ignoring NaNs)
    """
    excluded = set([c for c in df.columns if any(t in c.lower() for t in text_like)])
    candidates = []
    for c in df.columns:
        if c in excluded: 
            continue
        uniq = df[c].dropna().unique()
        # small set of uniques (exactly 2 preferred)
        if len(uniq) == 2:
            candidates.append(c)
    if candidates:
        return candidates[0]  # take the first; you can change this if needed
    # fallback: any small-cardinality column (<=3 uniques)
    for c in df.columns:
        if c in excluded: 
            continue
        uniq = df[c].dropna().unique()
        if 1 < len(uniq) <= 3:
            return c
    raise KeyError("Could not infer a binary label column. Please set label_col manually.")

# ---------- 1) ensure preprocessed_text exists ----------
text_col = ("preprocessed_text" 
            if "preprocessed_text" in data.columns 
            else detect_col(set(data.columns), ["EmailText","text","message","content","sms","Email","body"]))

if text_col is None:
    raise KeyError(f"No text column found. Available columns: {list(data.columns)}")

if text_col != "preprocessed_text":
    def basic_clean(s: str) -> str:
        s = str(s).lower()
        s = re.sub(r"http\S+|www\.\S+", " ", s)
        s = re.sub(r"@\w+|#\w+", " ", s)
        s = re.sub(r"(?is)<(script|style)[^>]*>.*?</\1>", " ", s)
        s = re.sub(r"(?is)<!--.*?-->", " ", s)
        s = re.sub(r"(?is)<[^>]+>", " ", s)
        s = re.sub(r"[^a-z\s]", " ", s)
        s = re.sub(r"\s+", " ", s).strip()
        return s
    data["preprocessed_text"] = data[text_col].fillna("").astype(str).apply(basic_clean)
text_col = "preprocessed_text"

# ---------- 2) find/normalize label column ----------
label_col = detect_col(set(data.columns), ["label","Label","target","y","spam","is_spam"])
if label_col is None:
    label_col = "label"

y_raw = data[label_col]

# Normalize to 0/1
if y_raw.dtype == object:
    y = y_raw.str.strip().str.lower().map({"ham": 0, "spam": 1})
    if y.isna().any():
        y, _ = pd.factorize(y_raw)
else:
    y = pd.to_numeric(y_raw, errors="coerce").astype("Int64")
    if y.dropna().nunique() != 2:
        y, _ = pd.factorize(y_raw)

# text features
X_text = data[text_col].fillna("").astype(str)

# ---------- 3) split ----------
X_train, X_val, y_train, y_val = train_test_split(
    X_text, y, test_size=0.2, random_state=42,
    stratify=y if pd.Series(y).dropna().nunique()>1 else None
)

print("Detected text column:", text_col)
print("Detected label column:", label_col)
print("Train/Val sizes:", len(X_train), len(X_val))
print("Label classes (train):", pd.Series(y_train).dropna().unique())


KeyError: 'label'

In [36]:
print(data.columns)
print(data.head(3).T)


Index(['text', 'preprocessed_text'], dtype='object')
                                                    0  \
text               I love natural language processing   
preprocessed_text  i love natural language processing   

                                                            1  \
text               Language processing is fun and interesting   
preprocessed_text  language processing is fun and interesting   

                                                          2  
text               I love coding in Python for data science  
preprocessed_text  i love coding in python for data science  


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [55]:
# Your code
#impact needed libraries
import re
import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# ====== 1) Detect columns & basic preprocessing ======
def detect_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    raise KeyError(f"None of these columns found among {list(cols)}: {candidates}")

# Detect text and label columns
text_col = ("preprocessed_text" if "preprocessed_text" in data.columns else
            detect_col(set(data.columns), ["EmailText","text","message","content","sms","Email","body"]))
label_col = detect_col(set(data.columns), ["label","Label","target","y","spam","is_spam"])

# Create preprocessed_text if needed
if text_col != "preprocessed_text":
    def basic_clean(s: str) -> str:
        s = str(s).lower()
        s = re.sub(r"http\S+|www\.\S+", " ", s)       # URLs
        s = re.sub(r"@\w+|#\w+", " ", s)              # mentions/hashtags
        s = re.sub(r"(?is)<(script|style)[^>]*>.*?</\1>", " ", s)  # inline JS/CSS
        s = re.sub(r"(?is)<!--.*?-->", " ", s)        # comments
        s = re.sub(r"(?is)<[^>]+>", " ", s)           # HTML tags
        s = re.sub(r"[^a-z\s]", " ", s)               # letters/spaces only
        s = re.sub(r"\s+", " ", s).strip()
        return s
    data["preprocessed_text"] = data[text_col].fillna("").astype(str).apply(basic_clean)
    text_col = "preprocessed_text"

# Normalize labels to 0/1
y_raw = data[label_col]
if y_raw.dtype == object:
    y = y_raw.str.lower().map({"ham":0, "spam":1})
    if y.isna().any():
        y, _ = pd.factorize(y_raw)   # fallback
else:
    y = y_raw.astype(int)

X_text = data[text_col].fillna("").astype(str)

# Train/val split
X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y if y.nunique()>1 else None
)

# ====== 2) Extra flags (money_mark, suspicious_words, text_len) ======
# We compute these ON THE FLY from raw preprocessed text and return a sparse matrix to keep NB happy.
money_pattern = re.compile(r"(?i)(euro|dollar|pound|€|\$|£)")
suspicious_re = re.compile(r"(?i)\b(free|cheap|sex|money|account|bank|fund|transfer|transaction|win|deposit|password)\b")

flag_names = np.array(["money_mark", "suspicious_words", "text_len"])

def flags_from_text(X_iterable):
    # X_iterable will be an array-like of strings
    mm, sw, tl = [], [], []
    for s in X_iterable:
        s = "" if s is None else str(s)
        mm.append(1 if money_pattern.search(s) else 0)
        sw.append(1 if suspicious_re.search(s) else 0)
        tl.append(len(s))
    arr = np.vstack([mm, sw, tl]).T  # shape (n_samples, 3)
    return sparse.csr_matrix(arr, dtype=np.float64)   # sparse to hstack nicely

flags_transformer = FunctionTransformer(flags_from_text, accept_sparse=False, validate=False)

# ====== 3) Build four feature configurations (always MultinomialNB with defaults) ======
# Common vectorizer settings that usually work well for spam/ham
bow = CountVectorizer(ngram_range=(1,2), min_df=3, max_features=30000, stop_words="english")
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_features=30000, stop_words="english")

# A) Bag of Words only
pipe_bow = Pipeline([
    ("vect", bow),
    ("clf", MultinomialNB())
])

# B) TF-IDF only
pipe_tfidf = Pipeline([
    ("vect", tfidf),
    ("clf", MultinomialNB())
])

# C) BoW + flags (FeatureUnion)
union_bow_flags = FeatureUnion([
    ("bow", bow),
    ("flags", flags_transformer),
])
pipe_bow_flags = Pipeline([
    ("features", union_bow_flags),
    ("clf", MultinomialNB())
])

# D) TF-IDF + flags (FeatureUnion)
union_tfidf_flags = FeatureUnion([
    ("tfidf", tfidf),
    ("flags", flags_transformer),
])
pipe_tfidf_flags = Pipeline([
    ("features", union_tfidf_flags),
    ("clf", MultinomialNB())
])

configs = [
    ("BoW only", pipe_bow),
    ("TF-IDF only", pipe_tfidf),
    ("BoW + flags", pipe_bow_flags),
    ("TF-IDF + flags", pipe_tfidf_flags),
]

# ====== 4) Fit, validate, and cross-validate (F1) ======
results = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, pipe in configs:
    # CV on full text+labels (uses only X_text here)
    cv_f1 = cross_val_score(pipe, X_text, y, cv=skf, scoring="f1").mean()
    # Train/val split scoring
    pipe.fit(X_train_text, y_train)
    y_pred = pipe.predict(X_val_text)
    acc = accuracy_score(y_val, y_pred)
    f1  = f1_score(y_val, y_pred)
    results.append((name, pipe, cv_f1, acc, f1))
    print(f"{name:15s} | 5-fold CV F1: {cv_f1:.3f} | Val Acc: {acc:.3f} | Val F1: {f1:.3f}")

# Pick best by validation F1 (tie-breaker CV F1)
best_name, best_pipe, best_cv_f1, best_acc, best_f1 = sorted(
    results, key=lambda t: (t[4], t[2]), reverse=True
)[0]

print(f"\nBest config: {best_name}")
print("\nValidation report:\n", classification_report(y_val, best_pipe.predict(X_val_text), digits=3))
print("Confusion matrix:\n", confusion_matrix(y_val, best_pipe.predict(X_val_text)))

# ====== 5) Show most relevant features for the best model ======
def get_feature_names_for_pipe(name, pipe):
    """Return feature names aligned with MultinomialNB input columns."""
    if name == "BoW only":
        return pipe.named_steps["vect"].get_feature_names_out()
    if name == "TF-IDF only":
        return pipe.named_steps["vect"].get_feature_names_out()
    if name == "BoW + flags":
        bow_names = pipe.named_steps["features"].transformer_list[0][1].get_feature_names_out()
        return np.concatenate([bow_names, flag_names])
    if name == "TF-IDF + flags":
        tfidf_names = pipe.named_steps["features"].transformer_list[0][1].get_feature_names_out()
        return np.concatenate([tfidf_names, flag_names])
    raise ValueError("Unknown configuration")

def top_nb_features(nb_clf, feat_names, top_k=20):
    """
    For MultinomialNB:
    feature_log_prob_ shape = (n_classes, n_features).
    We'll rank by log P(feature|spam) - log P(feature|ham).
    """
    logprob = nb_clf.feature_log_prob_
    if logprob.shape[0] != 2:
        # assume classes are [0,1] but check ordering
        # Sort by class index
        pass
    # Find index of ham (0) and spam (1) according to classes_
    classes = nb_clf.classes_
    if set(classes) != {0,1}:
        # Map class order to ham/spam indices
        cls_to_idx = {cls:i for i, cls in enumerate(classes)}
        ham_idx, spam_idx = cls_to_idx[0], cls_to_idx[1]
    else:
        # commonly [0,1]
        ham_idx, spam_idx = 0, 1

    score = logprob[spam_idx] - logprob[ham_idx]
    top_spam_idx = np.argsort(score)[-top_k:][::-1]
    top_ham_idx  = np.argsort(score)[:top_k]

    return (feat_names[top_spam_idx], score[top_spam_idx],
            feat_names[top_ham_idx],  score[top_ham_idx])

# Extract names + NB
if best_name in ["BoW only", "TF-IDF only"]:
    nb = best_pipe.named_steps["clf"]
else:
    nb = best_pipe.named_steps["clf"]

feat_names = get_feature_names_for_pipe(best_name, best_pipe)
top_spam_words, spam_scores, top_ham_words, ham_scores = top_nb_features(nb, feat_names, top_k=20)

print("\nTop features indicating SPAM:")
for w, s in zip(top_spam_words, spam_scores):
    print(f"{w:25s} {s: .3f}")

print("\nTop features indicating HAM:")
for w, s in zip(top_ham_words, ham_scores):
    print(f"{w:25s} {s: .3f}")


KeyError: "None of these columns found among ['preprocessed_text', 'text']: ['label', 'Label', 'target', 'y', 'spam', 'is_spam']"